In [15]:
import requests
import json
from IPython.display import JSON
from keys import *

In [16]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "fer261165"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [17]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [18]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [19]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [27]:
import pandas as pd

def flight_extraction(flight):
    return {
        'scheduled_arrival_time': flight.get('arrival', {}).get('scheduledTime', {}).get('local'),
        'flight_number': flight.get('number'),
        'from_airport': flight.get('departure', {}).get('airport', {}).get('name'),
        'airline': flight.get('airline', {}).get('name'),
        'aircraft': flight.get('aircraft', {}).get('model'),
        'icao_code': flight.get('departure', {}).get('airport', {}).get('icao', None)  # valor None si no existe
    }

# Crear DataFrame
flight_arrivals_df = pd.DataFrame([flight_extraction(f) for f in flight_arrivals.get('arrivals', [])])

# Convertir fecha a datetime sin zona horaria
flight_arrivals_df['scheduled_arrival_time'] = pd.to_datetime(
    flight_arrivals_df['scheduled_arrival_time'], errors='coerce'
).dt.tz_localize(None)

# Revisar datos antes de insertar
print(flight_arrivals_df.head())
print(flight_arrivals_df.info())

# Insertar en MySQL (asegúrate de tener la conexión 'con' lista)
flight_arrivals_df.to_sql('flight_arrival', if_exists='append', con=con, index=False)



  scheduled_arrival_time flight_number       from_airport           airline  \
0    2025-09-01 07:55:00       DE 4095  Frankfurt-am-Main            Condor   
1    2025-09-01 08:25:00       DE 2501              Dubai            Condor   
2    2025-09-01 08:35:00       PC 5067            Antalya           Pegasus   
3    2025-09-01 08:05:00       TP 8791           Helsinki  TAP Air Portugal   
4    2025-09-01 08:05:00       JL 6899           Helsinki                JL   

                  aircraft icao_code  
0              Airbus A320      EDDF  
1          Airbus A320 NEO      OMDB  
2  Airbus A320 (Sharklets)      LTAI  
3              Airbus A319      EFHK  
4              Airbus A319      EFHK  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   scheduled_arrival_time  320 non-null    datetime64[ns]
 1   fl

320

In [26]:
# Leer los ICAO existentes en la tabla airport
existing_icao = pd.read_sql("SELECT icao_code FROM airport", con=con)['icao_code'].tolist()

# Filtrar solo los nuevos ICAO
new_airports = flight_arrivals_df['icao_code'].dropna().unique()
new_airports = [icao for icao in new_airports if icao not in existing_icao]

# Crear DataFrame para insertar solo los ICAO nuevos
airport_insert_df = pd.DataFrame({
    'icao_code': new_airports,
    'airport_name': new_airports  # puedes reemplazar con nombres reales si los tienes
})

# Insertar en MySQL solo los nuevos
if not airport_insert_df.empty:
    airport_insert_df.to_sql('airport', if_exists='append', con=con, index=False)
